In [1]:
import pandas as pd
import numpy as np


In [2]:
business_capital = pd.read_excel("./business_capital_deals.xlsx",
                                parse_dates=True,
                                sheet_name="business_capital_deals",
                                usecols="A:G,K")

In [3]:
delta_capital = pd.read_excel("./delta_deals.xlsx",
                            parse_dates=True,
                            sheet_name="delta_deals",
                            usecols="A:G,K")

In [4]:
business_capital = business_capital.set_index(business_capital.created_at)

In [5]:
delta_capital = delta_capital.set_index(delta_capital.created_at)

In [6]:
delta_capital.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1628 entries, 2019-01-02 14:50:46 to 2019-10-29 21:48:58
Data columns (total 8 columns):
id                1628 non-null int64
name              1628 non-null object
created_at        1628 non-null datetime64[ns]
status            1628 non-null object
funded_amount     1628 non-null int64
expected_term     1628 non-null float64
monthly_volume    1349 non-null float64
underwriter       1628 non-null object
dtypes: datetime64[ns](1), float64(2), int64(2), object(3)
memory usage: 114.5+ KB


In [7]:
business_capital.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1682 entries, 2019-01-02 15:58:53 to 2019-10-29 20:56:09
Data columns (total 8 columns):
id                1682 non-null int64
name              1682 non-null object
created_at        1682 non-null datetime64[ns]
status            1682 non-null object
funded_amount     1682 non-null int64
expected_term     1682 non-null float64
monthly_volume    1226 non-null float64
underwriter       1682 non-null object
dtypes: datetime64[ns](1), float64(2), int64(2), object(3)
memory usage: 118.3+ KB


In [8]:
approved_statuses = ['Approved',
                     'Contracts Back',
                     'Contract Sent',
                     'Soft Approval',
                     'Ready to Fund',
                     'Lost Deal',
                     'Contract Returned',
                     'Funding Call',
                     'Open Approval']
declined_statuses = ['Negative Balances',
                     'Declined',
                     'Bad Credit',
                     'Too Small',
                     'No Room',
                     'Previous Default',
                     'Merchant Declined',
                     'Declined Previously',
                     'Too Few Deposits',
                     'SIC Code',
                     'Declined Bad Iso',
                     'Suspected Fraud',
                     'Auto-declined',
                     'Fraud',
                     'Missing Stips',
                     'Merchant Declined',
                     'No Logins',
                     'No COJ',
                     'Negative Banks',
                     'New MCA',
                     'Poor Landlord',
                     'Contracts Back Declined']
funded_statuses = ['Funded']

def status_groups(g):
    if g in approved_statuses:
        return "approved"
    elif g in declined_statuses:
        return "declined"
    elif g in funded_statuses:
        return "funded"
    else:
        return "still in submission"

In [9]:
business_capital['group_status'] = business_capital['status'].apply(status_groups)
delta_capital['group_status'] = delta_capital['status'].apply(status_groups)

In [10]:
business_capital.to_excel("abc.xlsx")

In [11]:
non_duplicates = business_capital[business_capital.duplicated(subset="name",keep=False)==False]

In [12]:
delta_capital.columns

Index(['id', 'name', 'created_at', 'status', 'funded_amount', 'expected_term',
       'monthly_volume', 'underwriter', 'group_status'],
      dtype='object')

In [13]:
delta_capital.groupby(pd.Grouper(freq='M'))["funded_amount"].sum()

created_at
2019-01-31    2012060
2019-02-28    2069000
2019-03-31    1954250
2019-04-30    1580000
2019-05-31    2357500
2019-06-30    2556000
2019-07-31    2904000
2019-08-31    2910000
2019-09-30    1620000
2019-10-31     634000
Freq: M, Name: funded_amount, dtype: int64

In [14]:
delta_capital.groupby(pd.Grouper(freq='M')).mean()

,id,funded_amount,expected_term,monthly_volume
created_at,,,,
2019-01-31,32110.698630,13781.232877,30.037600,168222.210317
2019-02-28,34468.834395,13178.343949,25.067178,243252.288028
2019-03-31,36750.776000,15634.000000,20.701591,158854.223214
2019-04-30,39463.808824,11617.647059,22.614245,190980.014824
2019-05-31,42078.867347,12028.061224,24.209233,141337.646438
2019-06-30,44698.266667,13107.692308,22.743473,179565.470909
2019-07-31,47466.166667,14235.294118,21.246178,150023.615917
2019-08-31,51633.875536,12489.270386,20.603251,228212.526162
2019-09-30,55371.629139,10728.476821,17.805637,169136.287445


In [15]:
month_counts = delta_capital.groupby([pd.Grouper(freq="M"), "group_status"])["funded_amount"].agg(["count","sum","mean"])

month_counts

count      sum          mean
created_at group_status                                     
2019-01-31 approved                67  1097060  16374.029851
           declined                54        0      0.000000
           funded                  20   915000  45750.000000
           still in submission      5        0      0.000000
2019-02-28 approved                67  1053000  15716.417910
           declined                64        0      0.000000
           funded                  18   974000  54111.111111
           still in submission      8    42000   5250.000000
2019-03-31 approved                53  1542500  29103.773585
           declined                53    50000    943.396226
           funded                  11   338750  30795.454545
           still in submission      8    23000   2875.000000
2019-04-30 approved                58   476000   8206.896552
           declined                54    80000   1481.481481
           funded                  16   903000  56437.500000
           still in submission      8   121000  15125.000000
2019-05-31 approved                83  1458500  17572.289157
           declined                82    25000    304.878049
           funded                  18   785000  43611.111111
           still in submission     13    89000   6846.153846
2019-06-30 approved                88  1270000  14431.818182
           declined                76    20000    263.157895
           funded                  21  1228000  58476.190476
           still in submission     10    38000   3800.000000
2019-07-31 approved                82  1343000  16378.048780
           declined                88   115000   1306.818182
           funded                  24  1311000  54625.000000
           still in submission     10   135000  13500.000000
2019-08-31 approved                77  1333000  17311.688312
           declined               124    24000    193.548387
           funded                  24  1520000  63333.333333
           still in submission      8    33000   4125.000000
2019-09-30 approved                52   535000  10288.461538
           declined                81        0      0.000000
           funded                  14  1057000  75500.000000
           still in submission      4    28000   7000.000000
2019-10-31 approved                30   231000   7700.000000
           declined                41    50000   1219.512195
           funded                   8   278000  34750.000000
           still in submission      6    75000  12500.000000

In [16]:
month_counts["pct_of_total_submissions"] = delta_capital.groupby([pd.Grouper(freq="M"), "group_status"])["funded_amount"].agg(["count"]).groupby(level=0).apply(lambda x: x/float(x.sum()))

month_counts["count_pct_ch"] = delta_capital.groupby([pd.Grouper(freq="M"),"group_status"])["id"].count().pct_change()
month_counts["sum_pct_ch"] = delta_capital.groupby([pd.Grouper(freq="M"),"group_status"])["funded_amount"].sum().pct_change()

# month_counts.to_excel("abc.xlsx")

In [17]:
month_counts = month_counts.unstack()

In [37]:
month_counts.columns

MultiIndex(levels=[['count', 'sum', 'mean', 'pct_of_total_submissions', 'count_pct_ch', 'sum_pct_ch'], ['approved', 'declined', 'funded', 'still in submission']],
           codes=[[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5], [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]],
           names=[None, 'group_status'])

In [58]:
attempt = month_counts[["count","sum"]]
attempt['change'] = attempt["count"].approved.pct_change()
attempt

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


count                                          sum           \
group_status approved declined funded still in submission approved declined   
created_at                                                                    
2019-01-31         67       54     20                   5  1097060        0   
2019-02-28         67       64     18                   8  1053000        0   
2019-03-31         53       53     11                   8  1542500    50000   
2019-04-30         58       54     16                   8   476000    80000   
2019-05-31         83       82     18                  13  1458500    25000   
2019-06-30         88       76     21                  10  1270000    20000   
2019-07-31         82       88     24                  10  1343000   115000   
2019-08-31         77      124     24                   8  1333000    24000   
2019-09-30         52       81     14                   4   535000        0   
2019-10-31         30       41      8                   6   231000    50000   

                                             change  
group_status   funded still in submission            
created_at                                           
2019-01-31     915000                   0       NaN  
2019-02-28     974000               42000  0.000000  
2019-03-31     338750               23000 -0.208955  
2019-04-30     903000              121000  0.094340  
2019-05-31     785000               89000  0.431034  
2019-06-30    1228000               38000  0.060241  
2019-07-31    1311000              135000 -0.068182  
2019-08-31    1520000               33000 -0.060976  
2019-09-30    1057000               28000 -0.324675  
2019-10-31     278000               75000 -0.423077

In [51]:
attempt["count"].columns

Index(['approved', 'declined', 'funded', 'still in submission'], dtype='object', name='group_status')

In [20]:
month_groups = delta_capital.groupby([pd.Grouper(freq='M')])["funded_amount"].agg(["count","sum","mean"])
month_groups["count_pct_ch"] = delta_capital.groupby(pd.Grouper(freq="M"))["id"].count().pct_change()
month_groups["sum_pct_ch"] = delta_capital.groupby(pd.Grouper(freq="M"))["funded_amount"].sum().pct_change()
month_groups

,count,sum,mean,count_pct_ch,sum_pct_ch
created_at,,,,,
2019-01-31,146,2012060,13781.232877,NaN,NaN
2019-02-28,157,2069000,13178.343949,0.075342,0.028299
2019-03-31,125,1954250,15634.000000,-0.203822,-0.055462
2019-04-30,136,1580000,11617.647059,0.088000,-0.191506
2019-05-31,196,2357500,12028.061224,0.441176,0.492089
2019-06-30,195,2556000,13107.692308,-0.005102,0.084199
2019-07-31,204,2904000,14235.294118,0.046154,0.136150
2019-08-31,233,2910000,12489.270386,0.142157,0.002066
2019-09-30,151,1620000,10728.476821,-0.351931,-0.443299


In [21]:
month_counts["pct_of_month's_submissions"] = delta_capital.groupby([pd.Grouper(freq='M'),'group_status'])["funded_amount"].agg("count").groupby(level=0).apply(lambda x: x / float(x.sum()))
month_counts

TypeError: incompatible index of inserted column with frame index